# Retrieve and save job results

{/*
  DO NOT EDIT THIS CELL!!!
  This cell's content is generated automatically by a script. Anything you add
  here will be removed next time the notebook is run. To add new content, create
  a new cell before or after this one.
*/}

<details>
<summary><b>Package versions</b></summary>

The code on this page was developed using the following requirements.
We recommend using these versions or newer.

```
qiskit-ibm-runtime~=0.43.1
```
</details>

Quantum workflows often take a while to complete and can run over many sessions. Restarting your Python kernel means you'll lose any results stored in memory. To avoid loss of data, you can save results to file and retrieve results of past jobs from IBM Quantum&reg; so your next session can continue where you left off.

## Retrieve job results from IBM Quantum

IBM Quantum automatically stores results from every job for you to retrieve at a later date. Use this feature to continue quantum programs across kernel restarts and review past results. You can get the ID of a job programmatically through its `job_id` method, or you can see all your submitted jobs and their IDs on the [Workloads page](https://quantum.cloud.ibm.com/workloads).

To find a job programmatically, use the [`QiskitRuntimeService.jobs`](/docs/api/qiskit-ibm-runtime/qiskit-runtime-service#jobs) method. By default, this returns the most recently submitted jobs, but you can also filter jobs by backend name, creation date, and more. The following cell finds any jobs submitted in the last three months. The `created_after` argument must be a [`datetime.datetime`](https://docs.python.org/3.8/library/datetime.html#datetime.datetime) object.

In [1]:
import datetime
from qiskit_ibm_runtime import QiskitRuntimeService

three_months_ago = datetime.datetime.now() - datetime.timedelta(days=90)

service = QiskitRuntimeService()
jobs_in_last_three_months = service.jobs(created_after=three_months_ago)
jobs_in_last_three_months[:3]  # show first three jobs

[<RuntimeJobV2('d5009i5eastc73cinf4g', 'sampler')>,
 <RuntimeJobV2('d5009i7g0u6s73db9dk0', 'estimator')>,
 <RuntimeJobV2('d50082vg0u6s73db9c10', 'sampler')>]

You can also select by backend, job state, session, and more. For more information, see [`QiskitRuntimeService.jobs`](/docs/api/qiskit-ibm-runtime/qiskit-runtime-service#jobs) in the API documentation.

Once you have the job ID, use the [`QiskitRuntimeService.job`](/docs/api/qiskit-ibm-runtime/qiskit-runtime-service#job) method to retrieve it.

In [2]:
# Get ID of most recent successful job for demonstration.
# This will not work if you've never successfully run a job.
successful_job = next(
    j for j in service.jobs(limit=1000) if j.status() == "DONE"
)
job_id = successful_job.job_id()
print(job_id)

qiskit_runtime_service._create_backend_obj:WARNING:2026-01-01 07:09:09,136: Unable to create configuration for ibm_brisbane. '404 Client Error: Not Found for url: https://us-east.quantum-computing.cloud.ibm.com/backends/ibm_brisbane/configuration. {"errors":[{"code":"not_found","message":"device not found","more_info":"https://cloud.ibm.com/apidocs/quantum-computing#error-handling"}],"trace":"a9bf13a7-b763-4b66-af1c-1db3a5418ffa"}\n' 


qiskit_runtime_service._create_backend_obj:WARNING:2026-01-01 07:09:09,267: Unable to create configuration for ibm_brisbane. '404 Client Error: Not Found for url: https://us-east.quantum-computing.cloud.ibm.com/backends/ibm_brisbane/configuration. {"errors":[{"code":"not_found","message":"device not found","more_info":"https://cloud.ibm.com/apidocs/quantum-computing#error-handling"}],"trace":"67d568ff-e73d-42c8-aa0e-8c2a19523062"}\n' 


qiskit_runtime_service._create_backend_obj:WARNING:2026-01-01 07:09:09,410: Unable to create configuration for ibm_brisbane. '404 Client Error: Not Found for url: https://us-east.quantum-computing.cloud.ibm.com/backends/ibm_brisbane/configuration. {"errors":[{"code":"not_found","message":"device not found","more_info":"https://cloud.ibm.com/apidocs/quantum-computing#error-handling"}],"trace":"1cc24e0e-9d25-4850-a4fa-a4df08196812"}\n' 


d5009i5eastc73cinf4g


In [3]:
retrieved_job = service.job(job_id)
retrieved_job.result()

PrimitiveResult([SamplerPubResult(data=DataBin(meas=BitArray(<shape=(3, 2), num_shots=4096, num_bits=2>), meas0=BitArray(<shape=(3, 2), num_shots=4096, num_bits=133>), shape=(3, 2)), metadata={'circuit_metadata': {}})], metadata={'execution': {'execution_spans': ExecutionSpans([DoubleSliceSpan(<start='2025-12-15 14:14:30', stop='2025-12-15 14:14:38', size=24576>)])}, 'version': 2})

<CodeAssistantAdmonition
  tagLine="Always forgetting how to retrieve a job result? Try this prompt with Qiskit Code Assistant:"
  prompts={[
    "# Retrieve job JOB_ID from IBM Runtime and print the result"
  ]}
/>

## Save results to disk

You may also want to save results to disk. To do this, use Python's built-in JSON library with encoders from Qiskit Runtime.

In [4]:
import json
from qiskit_ibm_runtime import RuntimeEncoder

with open("result.json", "w") as file:
    json.dump(retrieved_job.result(), file, cls=RuntimeEncoder)

You can then load this array from disk in a separate kernel.

In [5]:
from qiskit_ibm_runtime import RuntimeDecoder

with open("result.json", "r") as file:
    result = json.load(file, cls=RuntimeDecoder)

result

PrimitiveResult([SamplerPubResult(data=DataBin(meas=BitArray(<shape=(3, 2), num_shots=4096, num_bits=2>), meas0=BitArray(<shape=(3, 2), num_shots=4096, num_bits=133>), shape=(3, 2)), metadata={'circuit_metadata': {}})], metadata={'execution': {'execution_spans': ExecutionSpans([DoubleSliceSpan(<start='2025-12-15 14:14:30', stop='2025-12-15 14:14:38', size=24576>)])}, 'version': 2})